In [ ]:
import pandas as pd
from predictDepth import *
from zedStereoUtility import *

image_dir = './dataset/1642520848/'
left_image_file = os.path.join(image_dir, 'left', 'frame015.jpg')
right_image_file = os.path.join(image_dir, 'right', 'frame015.jpg')
left_image = np.array(Image.open(left_image_file))
right_image = np.array(Image.open(right_image_file))

### 1. Generate Disparity Map

In [ ]:
disp_map = getDisparityMap(left_image, right_image)

### 2. Load Analysed pickle file

In [ ]:
object_set = 1642520848
l_datafile = '../camera-main/videos/{}/{}-leftDLC_dlcrnetms5_make_tea_01052022Jan5shuffle1_40000_full.pickle'.format(object_set, object_set)
with open(l_datafile, 'rb') as file:
    l_data = pkl.load(file)
    
# Getting the best pixels from the file.
best_pixels = []
frame_coord = l_data['frame0015']
coord, confid = frame_coord['coordinates'][0], frame_coord['confidence']
for k, _ in enumerate(coord):
    confid_k = list(np.reshape(confid[k], (-1)))
    best_index = confid_k.index(max(confid_k))
    best_pixels.append(coord[k][best_index])

# Use LEAStereoCoordinate if there is a image and disparity directory.

In [ ]:
px, py = [i[0] for i in best_pixels] , [i[1] for i in best_pixels] 
fig = plt.figure(figsize=(15, 25))
fig.add_subplot(1, 2, 1)
plt.title('Left Image')
plt.scatter(px, py)
plt.imshow(left_image)
fig.add_subplot(1, 2, 2)
plt.title('Disparity')
plt.imshow(disp_map)

### 3. Convert Image and Disparity to 3D Coordinates

In [ ]:
coordinate3D = pixelTo3DCameraCoord(left_image, disp_map, best_pixels)

In [ ]:
import plotly.express as px
XYZ = [[i['X'] for i in coordinate3D], [i['Y'] for i in coordinate3D], [i['Z'] for i in coordinate3D]]
XYZ = list(zip(*XYZ))
tb_dlc_df = pd.DataFrame(np.array(XYZ))
tb_dlc_df.columns = ['X', 'Y', 'Z']
fig = px.scatter_3d(tb_dlc_df,x='X',y='Y',z='Z')
fig.update_traces(marker=dict(size=1))
fig.show()